In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2
import vpype
from skimage import io
from pathlib import Path

from sklearn.preprocessing import minmax_scale
from skimage import feature
from genpen.utils import Paper

In [ ]:
def polygonize_circle(circle, n_corners):
    angles = np.linspace(0, 1, n_corners+1)
    corners = [circle.boundary.interpolate(a, normalized=True) for a in angles]
    return Polygon(corners)

def reg_polygon(point, radius, n_corners):
    circle = point.buffer(radius)
    return polygonize_circle(circle, n_corners)

@dataclass
class RegPolygon(object):
    point: Point
    radius: float=1.
    n_corners: int = 6
    rotation: float = 0.  #degrees
   
    @property
    def poly(self):
        poly = reg_polygon(self.point, self.radius, self.n_corners)
        return sa.rotate(poly, self.rotation)
    
    @property
    def corners(self):
        return MultiPoint(self.poly.boundary.coords)[:-1]

In [ ]:
@dataclass
class StellarSnowflake(object):
    point_f0: Point = Point((0,0))
    radius_f0: float=1.
    n_corners_f0: int = 6
    rotation_f0: float = 0.  #degrees
    radius_f1: float=0.3
    n_corners_f1: int = 6
    rotation_f1: float = 0.  #degrees    
    
    @property
    def f0(self):
        return RegPolygon(self.point_f0,
                          self.radius_f0, self.n_corners_f0, self.rotation_f0)
    
    @property
    def poly_center(self):
        return self.f0.poly
    
    @property
    def corner_polys(self):
        corner_polys = []
        for p in self.f0.corners:
            cp = RegPolygon(point=p, 
                             radius=self.radius_f1, 
                             n_corners=self.n_corners_f1,
                             rotation=self.rotation_f1,
                           ).poly
            corner_polys.append(cp) 
        return MultiPolygon(corner_polys)
    
    @property
    def multipolygon(self):
        return gp.merge_Polygons([self.poly_center, self.corner_polys])
    
    @property
    def poly(self):
        return so.unary_union([self.poly_center, self.corner_polys])

In [ ]:
def morsify(ls, buffer_factor=0.01, ):
    dilated = ls.buffer(buffer_factor)
    eroded = dilated.buffer(-buffer_factor)
    return eroded.boundary

In [ ]:
# make page
paper_size = '5x7 inches'
border:float=20
paper = Paper('7x5 inches')

drawbox = paper.get_drawbox(border)

In [ ]:
xs, ys = gp.overlay_grid(drawbox, xstep=55, ystep=200)

In [ ]:
points = [Point(x,y) for x,y in itertools.product(xs,ys)]

In [ ]:
layers = []
n_layers = 5

for i in range(n_layers):
    fills = []
    for point in points:
        radius_f0=15
        f0f1_rad_ratio = np.random.uniform(0.25, 0.65)
        radius_f1 = radius_f0 * f0f1_rad_ratio
        rotation_f0 = np.random.uniform(0,180)
        rotation_f1 = np.random.uniform(0,180)
        sf = StellarSnowflake(
            point, 
            radius_f0=radius_f0, 
            radius_f1=radius_f1,
            rotation_f0 = rotation_f0,
            rotation_f1 = rotation_f1,
        )

        poly_center = sf.poly_center.buffer(-1).buffer(1).buffer(np.random.uniform(-2,-0.5))
        corner_polys = sf.corner_polys.buffer(-1).buffer(1).buffer(np.random.uniform(0.2, 1))
        p = so.unary_union([poly_center, corner_polys])

        hole = RegPolygon(point, radius=np.random.uniform(1,5), n_corners=6, 
                          rotation=np.random.uniform(0,180)).poly
        p = p.difference(hole)


        for i in range(1):
            d = np.random.uniform(0.5, 2)
            p = p.buffer(-d).buffer(d)


        stp = gp.ScaleTransPrms(d_buffer=-0.35,angles=-45,d_translate_factor=0.)
        P = gp.Poly(p)
        P.fill_scale_trans(**stp.prms)

        buffer_factor = 1e-4
        cs=1
        js=1
        r = 27
        mfills = []
        for f in P.fill:
            dilated = f.buffer(buffer_factor, cap_style=cs, join_style=js, resolution=r)
            eroded = dilated.buffer(-buffer_factor, cap_style=cs, join_style=js, resolution=r)
            mfills.append(eroded.boundary)

        mfills = gp.merge_LineStrings(mfills)
        mfills = gp.merge_LineStrings([f for f in mfills if f.length>0.3])
        fills.append(mfills)
        
    
    fill_layer = gp.merge_LineStrings(fills)
    layers.append(fill_layer)

In [ ]:

sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

tolerance=0.5
sk.vpype(f'linemerge --tolerance {tolerance}mm linesort')

sk.display()

In [ ]:
savedir='/mnt/c/code/side/plotter_images/oned_outputs'
filename = '0148_imperfect_snowflakes_thinpack.svg'

In [ ]:
savepath = Path(savedir).joinpath(filename).as_posix()
sk.save(savepath)